In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [18]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [19]:
from fastai.text.all import *

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
%cd /content/drive/MyDrive/Capstone3

/content/drive/MyDrive/Capstone3


In [50]:
df = pd.read_csv("imdb_movies.csv")
df.head()

,movie title,Run Time,Rating,User Rating,Generes,Overview,Plot Kyeword,Director,Top 5 Casts,Writer,year,path
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"['Action', 'Drama']","After more than thirty years of service as one of the Navy's top aviators, Pete Mitchell is where he belongs, pushing the envelope as a courageous test pilot and dodging the advancement in rank that would ground","['fighter jet', 'sequel', 'u.s. navy', 'fighter aircraft', 'pilot', 'naval aviation', 'jet fighter', 'military pilot', 'aviation', 'aircraft']",Joseph Kosinski,"['Jack Epps Jr.', 'Peter Craig', 'Tom Cruise', 'Jennifer Connelly', 'Miles Teller']",Jim Cash,-2022,/title/tt1745960/
1,Jurassic World Dominion,2 hours 27 minutes,6,56K,"['Action', 'Adventure', 'Sci-Fi']","Four years after the destruction of Isla Nublar, dinosaurs now live--and hunt--alongside humans all over the world. This fragile balance will reshape the future and determine, once and for all, whether human beings are to remain the apex predators on a planet they now share with ... Read all","['dinosaur', 'jurassic park', 'tyrannosaurus rex', 'velociraptor', 'sequel', 'alan grant character', 'ellie sattler character', 'ian malcolm character', 'dinosaur feature', 'animal trainer']",Colin Trevorrow,"['Colin Trevorrow', 'Derek Connolly', 'Chris Pratt', 'Bryce Dallas Howard', 'Laura Dern']",Emily Carmichael,-2022,/title/tt8041270/
2,Top Gun,"$15,000,000 (estimated)",6.9,380K,"['Action', 'Drama']","As students at the United States Navy's elite fighter weapons school compete to be best in the class, one daring young pilot learns a few things from a civilian instructor that are not taught in the classroom.","['pilot', 'male camaraderie', 'u.s. navy', 'grumman f 14 tomcat', 'flying', 'jet fighter', 'fighter pilot', 'jet', 'rivalry', 'flight school']",Tony Scott,"['Jack Epps Jr.', 'Ehud Yonay', 'Tom Cruise', 'Tim Robbins', 'Kelly McGillis']",Jim Cash,-1986,/title/tt0092099/
3,Lightyear,"$71,101,257",5.2,32K,"['Animation', 'Action', 'Adventure']","While spending years attempting to return home, marooned Space Ranger Buzz Lightyear encounters an army of ruthless robots commanded by Zurg who are attempting to steal his fuel source.","['galaxy', 'spaceship', 'robot', 'rocket', 'space adventure', 'same sex kiss', 'brainwashing', 'anti conservative', 'spin off', 'pixar']",Angus MacLane,"['Jason Headley', 'Matthew Aldrich', 'Chris Evans', 'Keke Palmer', 'Peter Sohn']",Angus MacLane,-2022,/title/tt10298810/
4,Spiderhead,not-released,5.4,23K,"['Action', 'Crime', 'Drama']","In the near future, convicts are offered the chance to volunteer as medical subjects to shorten their sentence. One such subject for a new drug capable of generating feelings of love begins questioning the reality of his emotions.","['discover', 'medical', 'test', 'reality', 'fictional drug', 'violence', 'room', 'nature', 'emotion', 'volunteer']",Joseph Kosinski,"['Rhett Reese', 'Paul Wernick', 'Chris Hemsworth', 'Miles Teller', 'Jurnee Smollett']",George Saunders,-2022,/title/tt9783600/


In [51]:
df.shape

(24402, 12)

In [52]:
df.isna().sum()

movie title       0
Run Time          0
Rating            0
User Rating       0
Generes           0
Overview        244
Plot Kyeword      0
Director          0
Top 5 Casts       0
Writer            0
year            778
path              0
dtype: int64

In [53]:
df = df.dropna().reset_index(drop=True)

In [68]:
genre_list = []
indices_to_drop = []

for idx,Genres in enumerate(df.Generes.to_list()):
  generes_list = eval(Genres)

  if len(generes_list):
    generes_list.append(generes_list[0])
  else:
    indices_to_drop.append(idx)

df = df.drop(indices_to_drop).reset_index(drop=True)
df.shape

(23412, 12)

In [69]:
genre_count = df['Generes'].value_counts()
threshold = int(len(df) * 0.005)
rare_genres = [ cat for cat, count in genre_count.items() if count <threshold]
len(rare_genres), rare_genres[:5]

(686,
 ["['Action', 'Adventure', 'Thriller']",
  "['Crime', 'Mystery', 'Thriller']",
  "['Action', 'Sci-Fi', 'Thriller']",
  "['Action', 'Sci-Fi']",
  "['Horror', 'Sci-Fi', 'Thriller']"])

In [70]:
rare_indices_to_drop = [idx for idx, row in df.iterrows() if row['Generes'] in rare_genres]
len(rare_indices_to_drop)

9249

In [71]:
df = df.drop(rare_indices_to_drop).reset_index(drop=True)
df.shape

(14163, 12)

In [72]:
df['Generes'].value_counts()

['Drama']                               895
['Action', 'Crime', 'Drama']            844
['Comedy', 'Drama', 'Romance']          606
['Crime', 'Drama', 'Thriller']          600
['Crime', 'Drama']                      530
['Comedy', 'Drama']                     527
['Drama', 'Romance']                    502
['Action', 'Adventure', 'Comedy']       395
['Comedy']                              377
['Action']                              374
['Action', 'Thriller']                  371
['Horror', 'Thriller']                  368
['Crime', 'Drama', 'Mystery']           367
['Action', 'Comedy', 'Crime']           360
['Drama', 'Thriller']                   357
['Action', 'Crime', 'Thriller']         353
['Action', 'Drama']                     345
['Comedy', 'Romance']                   344
['Action', 'Adventure', 'Drama']        326
['Animation', 'Adventure', 'Comedy']    312
['Comedy', 'Crime', 'Drama']            309
['Action', 'Drama', 'Thriller']         286
['Thriller']                    

In [73]:
len(df['Generes'].value_counts())

47

In [74]:
df.describe(include='object')

,movie title,Run Time,Rating,User Rating,Generes,Overview,Plot Kyeword,Director,Top 5 Casts,Writer,year,path
count,14163,14163,14163,14163,14163,14163,14163,14163,14163,14163,14163,14163
unique,13958,993,88,1542,47,14134,12767,7901,14127,10106,227,13958
top,Rage,not-released,no-rating,0,['Drama'],none,[],Woody Allen,"['See producer', 'See preliminary cast']",Woody Allen,-2022,/title/tt0114224/
freq,4,4558,556,556,895,18,828,33,18,33,756,4


In [75]:
doc(RandomSplitter)

RandomSplitter(valid_pct=0.2, seed=None)
Create function that splits `items` between train/val with `valid_pct` randomly.

To get a prettier result with hyperlinks to source code and documentation, install nbdev: pip install nbdev
